In [1]:
import pandas as pd, numpy as np

# Chargement des sources
uploadées sur google drive

In [2]:
%ls

drive/  sample_data/


In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/sources/TradingView_strategies/with_fees/
%ls

/content/drive/MyDrive/Colab Notebooks/sources/TradingView_strategies/with_fees
1000_max_Profit_Factor_Long_2006-2023.csv
2000_max_Profit_Factor_Long_1999-2023_failed.csv
2000_max_Profit_Factor_Short_2006-2023.csv
500_max_Net_Profit_Long_1999-2023.csv


In [7]:
sources = ['1000_max_Profit_Factor_Long_2006-2023.csv',
'2000_max_Profit_Factor_Long_1999-2023_failed.csv',
'2000_max_Profit_Factor_Short_2006-2023.csv',
'500_max_Net_Profit_Long_1999-2023.csv']
print(sources)

['1000_max_Profit_Factor_Long_2006-2023.csv', '2000_max_Profit_Factor_Long_1999-2023_failed.csv', '2000_max_Profit_Factor_Short_2006-2023.csv', '500_max_Net_Profit_Long_1999-2023.csv']


In [8]:
path = '/content/drive/MyDrive/Colab Notebooks/sources/TradingView_strategies/with_fees/'
df_all_tests = pd.read_csv(path+sources[0])

In [9]:
# Ajout de la colonne portant le nom du fichier, pour préciser le contexte des résultats obtenus
nom_test = str.split(sources[0],'.')
df_all_tests["Test"] = nom_test[0]

In [10]:
df_all_tests.head(1)

,Net Profit: All,Net Profit %: All,Net Profit: Long,Net Profit %: Long,Net Profit: Short,Net Profit %: Short,Gross Profit: All,Gross Profit %: All,Gross Profit: Long,Gross Profit %: Long,...,__Date debut Backtesting,__Date fin Backtesting,__AlphaTrend Period,__AlphaTrend Multiplier,__Periode ATR utilisée Stop Loss / Take Profit,__ATR Stop Loss coefficient,__Ratio TP:SL soit 1:x,__Show Signals?,__Change calculation (no volume data)?,Test
0,61.88,6.19,639.59,63.96,-577.7,-57.77,6627.91,662.79,5620.55,562.06,...,2006.0,2023.0,32.0,1.0,14.0,2.0,3.0,True,True,1000_max_Profit_Factor_Long_2006-2023


In [11]:
for i in range (1,len(sources)):
  '''On commence à indice 1 car df_all_tests est constitué avec le 1er élément de la liste 'sources'''
  
  df_concat = pd.read_csv(path+sources[i])

  # Ajout de la colonne portant le nom du fichier, pour préciser le contexte des résultats obtenus
  nom_test = str.split(sources[i],'.')
  # On conserve du split uniquement le 1er élement, c'est à dire le nom du fichier
  df_concat["Test"] = nom_test[0]

  df_all_tests = pd.concat([df_all_tests,df_concat])

In [12]:
print("Nombre de tests réalisés au total : ",df_all_tests.shape[0])

Nombre de tests réalisés au total :  5121


# Exploitation des résultats

In [13]:
all_cols = df_all_tests.columns.to_list()
all_cols

['Net Profit: All',
 'Net Profit %: All',
 'Net Profit: Long',
 'Net Profit %: Long',
 'Net Profit: Short',
 'Net Profit %: Short',
 'Gross Profit: All',
 'Gross Profit %: All',
 'Gross Profit: Long',
 'Gross Profit %: Long',
 'Gross Profit: Short',
 'Gross Profit %: Short',
 'Gross Loss: All',
 'Gross Loss %: All',
 'Gross Loss: Long',
 'Gross Loss %: Long',
 'Gross Loss: Short',
 'Gross Loss %: Short',
 'Max Run-up',
 'Max Run-up %',
 'Max Drawdown',
 'Max Drawdown %',
 'Buy & Hold Return',
 'Buy & Hold Return %',
 'Sharpe Ratio',
 'Sortino Ratio',
 'Profit Factor: All',
 'Profit Factor: Long',
 'Profit Factor: Short',
 'Max Contracts Held: All',
 'Max Contracts Held: Long',
 'Max Contracts Held: Short',
 'Open PL',
 'Open PL %',
 'Commission Paid: All',
 'Commission Paid: Long',
 'Commission Paid: Short',
 'Total Closed Trades: All',
 'Total Closed Trades: Long',
 'Total Closed Trades: Short',
 'Total Open Trades: All',
 'Total Open Trades: Long',
 'Total Open Trades: Short',
 'Numb

In [16]:
short_cols_selection = [
'Net Profit %: All',
'Net Profit %: Long',
'Net Profit %: Short',
'Max Drawdown %',
'Profit Factor: All',
'Profit Factor: Long',
'Profit Factor: Short',
'Sortino Ratio',
'Percent Profitable: All',
'Percent Profitable: Long',
'Percent Profitable: Short',
'Total Closed Trades: All',
'Total Closed Trades: Long',
'Total Closed Trades: Short',
'__Moyenne Mobile Exponentielle',
'__STC length',
'__Fast length',
'__Slow length',
'__AlphaTrend Period',
'__AlphaTrend Multiplier',
'__ATR Stop Loss coefficient',
'__Ratio TP:SL soit 1:x',
'Test']

In [17]:
df_light_all_tests = df_all_tests[short_cols_selection]
df_light_all_tests.reset_index(drop=True, inplace=True)

In [36]:
# Ne conserve que les n premières lignes /catégories
indexes = df_light_all_tests.groupby(by=["Test"])["Net Profit %: Long"].nlargest(1)

# Transforme la serie avec multi index en DataFrame
df_indexes = indexes.index.to_frame()

df_indexes.reset_index(drop=True, inplace=True)
df_indexes.columns=["Test","Index_best"]

In [37]:
best_rows = df_indexes["Index_best"]
print(best_rows)

0     648
1     954
2    4261
3    5110
Name: Index_best, dtype: int64


In [38]:
df_best_net_profit_Long = df_light_all_tests.iloc[best_rows]
df_best_net_profit_Long.sort_values(by=["Net Profit %: Long"], ascending=False)

,Net Profit %: All,Net Profit %: Long,Net Profit %: Short,Max Drawdown %,Profit Factor: All,Profit Factor: Long,Profit Factor: Short,Sortino Ratio,Percent Profitable: All,Percent Profitable: Long,...,Total Closed Trades: Short,__Moyenne Mobile Exponentielle,__STC length,__Fast length,__Slow length,__AlphaTrend Period,__AlphaTrend Multiplier,__ATR Stop Loss coefficient,__Ratio TP:SL soit 1:x,Test
648,687.07,712.42,-25.35,105.14,2.22,3.92,0.92,0.26,56.67,67.50,...,20.0,45.0,95.0,35.0,70.0,36.0,0.2,2.0,3.0,1000_max_Profit_Factor_Long_2006-2023
5110,632.54,680.63,-48.09,143.28,1.85,2.86,0.87,-0.07,48.89,61.40,...,33.0,75.0,105.0,25.0,105.0,36.0,0.2,2.0,3.0,500_max_Net_Profit_Long_1999-2023
4261,514.57,566.88,-52.31,205.07,1.61,2.24,0.87,-0.09,46.32,56.67,...,35.0,40.0,105.0,30.0,70.0,36.0,0.2,2.0,3.0,2000_max_Profit_Factor_Short_2006-2023
954,196.66,260.52,-63.86,144.37,1.32,1.63,0.69,-0.02,47.37,53.70,...,22.0,195.0,100.0,20.0,70.0,32.0,0.4,2.0,3.0,2000_max_Profit_Factor_Long_1999-2023_failed


Meilleurs (plus faibles) Drawdonws

In [40]:
# Attention : nsmallest pour Max Drawdown
indexes = df_light_all_tests.groupby(by=["Test"])["Net Profit %: Short"].nlargest(1)

# Transforme la serie avec multi index en DataFrame
df_indexes = indexes.index.to_frame()

df_indexes.reset_index(drop=True, inplace=True)
df_indexes.columns=["Test","Index_best"]

# Index des meilleurs lignes retenues
best_rows = df_indexes["Index_best"]

In [41]:
df_best_net_profit_short = df_light_all_tests.iloc[best_rows]
df_best_net_profit_short.sort_values(by=["Net Profit %: Short"], ascending=False)

,Net Profit %: All,Net Profit %: Long,Net Profit %: Short,Max Drawdown %,Profit Factor: All,Profit Factor: Long,Profit Factor: Short,Sortino Ratio,Percent Profitable: All,Percent Profitable: Long,...,Total Closed Trades: Short,__Moyenne Mobile Exponentielle,__STC length,__Fast length,__Slow length,__AlphaTrend Period,__AlphaTrend Multiplier,__ATR Stop Loss coefficient,__Ratio TP:SL soit 1:x,Test
4855,178.98,18.77,160.20,99.77,1.20,1.03,1.72,0.25,46.25,50.94,...,27.0,45.0,105.0,25.0,105.0,36.0,1.2,2.0,3.0,500_max_Net_Profit_Long_1999-2023
4202,144.31,-12.73,157.04,111.86,1.15,0.98,1.70,0.81,47.62,51.72,...,26.0,40.0,100.0,30.0,70.0,36.0,1.2,2.0,3.0,2000_max_Profit_Factor_Short_2006-2023
797,223.30,86.47,136.83,224.71,1.35,1.17,2.19,-0.03,46.94,48.78,...,8.0,195.0,80.0,20.0,70.0,32.0,1.0,2.0,3.0,1000_max_Profit_Factor_Long_2006-2023
1077,159.05,60.79,98.27,279.06,1.20,1.10,1.56,-0.07,43.24,48.21,...,18.0,195.0,80.0,20.0,70.0,32.0,1.0,2.0,3.0,2000_max_Profit_Factor_Long_1999-2023_failed


Meilleurs Sortino Ratio

In [32]:
# Attention : nsmallest pour Max Drawdown
indexes = df_light_all_tests.groupby(by=["Test"])["Profit Factor: Long"].nlargest(1)

# Transforme la serie avec multi index en DataFrame
df_indexes = indexes.index.to_frame()

df_indexes.reset_index(drop=True, inplace=True)
df_indexes.columns=["Test","Index_best"]

# Index des meilleurs lignes retenues
best_rows = df_indexes["Index_best"]

In [33]:
df_best_Profit_Factor_Long = df_light_all_tests.iloc[best_rows]
df_best_Profit_Factor_Long.sort_values(by=["Profit Factor: Long"], ascending=False)

,Net Profit %: All,Net Profit %: Long,Net Profit %: Short,Max Drawdown %,Profit Factor: All,Profit Factor: Long,Profit Factor: Short,Sortino Ratio,Percent Profitable: All,Percent Profitable: Long,...,Total Closed Trades: Short,__Moyenne Mobile Exponentielle,__STC length,__Fast length,__Slow length,__AlphaTrend Period,__AlphaTrend Multiplier,__ATR Stop Loss coefficient,__Ratio TP:SL soit 1:x,Test
390,611.15,662.06,-50.92,86.06,2.57,5.62,0.79,0.24,59.18,70.59,...,15.0,100.0,95.0,35.0,70.0,36.0,0.2,2.0,3.0,1000_max_Profit_Factor_Long_2006-2023
4865,649.29,677.24,-27.95,143.28,2.01,3.32,0.92,-0.07,49.43,61.82,...,32.0,80.0,105.0,25.0,105.0,36.0,0.2,2.0,3.0,500_max_Net_Profit_Long_1999-2023
4261,514.57,566.88,-52.31,205.07,1.61,2.24,0.87,-0.09,46.32,56.67,...,35.0,40.0,105.0,30.0,70.0,36.0,0.2,2.0,3.0,2000_max_Profit_Factor_Short_2006-2023
954,196.66,260.52,-63.86,144.37,1.32,1.63,0.69,-0.02,47.37,53.70,...,22.0,195.0,100.0,20.0,70.0,32.0,0.4,2.0,3.0,2000_max_Profit_Factor_Long_1999-2023_failed


Meilleurs Profit Factors

In [34]:
# Attention : nsmallest pour Max Drawdown
indexes = df_light_all_tests.groupby(by=["Test"])["Profit Factor: Short"].nlargest(1)

# Transforme la serie avec multi index en DataFrame
df_indexes = indexes.index.to_frame()

df_indexes.reset_index(drop=True, inplace=True)
df_indexes.columns=["Test","Index_best"]

# Index des meilleurs lignes retenues
best_rows = df_indexes["Index_best"]

In [35]:
df_best_Profit_Factor_Short = df_light_all_tests.iloc[best_rows]
df_best_Profit_Factor_Short.sort_values(by=["Profit Factor: Short"], ascending=False)

,Net Profit %: All,Net Profit %: Long,Net Profit %: Short,Max Drawdown %,Profit Factor: All,Profit Factor: Long,Profit Factor: Short,Sortino Ratio,Percent Profitable: All,Percent Profitable: Long,...,Total Closed Trades: Short,__Moyenne Mobile Exponentielle,__STC length,__Fast length,__Slow length,__AlphaTrend Period,__AlphaTrend Multiplier,__ATR Stop Loss coefficient,__Ratio TP:SL soit 1:x,Test
2799,48.47,-54.27,102.74,129.16,1.11,0.87,9.95,2.51,48.72,48.39,...,8.0,40.0,105.0,30.0,70.0,36.0,2.6,2.0,3.0,2000_max_Profit_Factor_Short_2006-2023
4812,29.06,-66.64,95.70,151.86,1.07,0.84,6.17,-0.07,44.74,44.83,...,9.0,45.0,95.0,25.0,105.0,36.0,2.6,2.0,3.0,500_max_Net_Profit_Long_1999-2023
904,-69.41,-118.87,49.46,161.34,0.85,0.74,5.13,0.02,44.44,45.83,...,3.0,100.0,95.0,35.0,70.0,38.0,2.6,2.0,3.0,1000_max_Profit_Factor_Long_2006-2023
961,140.16,91.26,48.90,138.70,1.33,1.25,1.75,-0.02,47.62,54.84,...,11.0,195.0,100.0,20.0,70.0,32.0,2.0,2.0,3.0,2000_max_Profit_Factor_Long_1999-2023_failed


# Agrégation et export

In [42]:
df_best_aggregate = pd.concat([df_best_net_profit_Long, df_best_net_profit_short, df_best_Profit_Factor_Long, df_best_Profit_Factor_Short])

In [43]:
df_best_aggregate.to_excel(path+"best_aggregate_with_fees_20230406.xlsx", header=True, index=False)